<h1 id='dataset-merger' style='color:#7159c1; border-bottom:3px solid #7159c1; letter-spacing:2px; font-family:JetBrains Mono; font-weight: bold; text-align:center; font-size:240%;padding:0'>📝 Dataset Merger</h1>

<h1 id='0-settings' style='color:#7159c1; border-bottom:3px solid #7159c1; letter-spacing:2px; font-family:JetBrains Mono; font-weight: bold; text-align:left; font-size:240%;padding:0'>0 | Settings</h1>

In [3]:
# ---- Packages ----
import numpy as np # pip install numpy
import pandas as pd # pip install pandas

# ---- Constants ----
DATASETS_PATH = ('./datasets')

# ---- Functions ----
split_to = lambda price, threshold=200, split=10: price / split if price >= threshold else price

<h1 id='1-preparing-datasets' style='color:#7159c1; border-bottom:3px solid #7159c1; letter-spacing:2px; font-family:JetBrains Mono; font-weight: bold; text-align:left; font-size:240%;padding:0'>1 | Preparing Datasets</h1>

In [4]:
# ---- HABT11 Dataset ----
habt_df = pd.read_csv(f'{DATASETS_PATH}/Dataset - HABT11.csv')
habt_df.rename(columns={"Date": "date", "Close": "habt"}, inplace=True)
habt_df.date = pd.to_datetime(habt_df.date, format='%d/%m/%Y %H:%M:%S')
habt_df.set_index(keys=['date'], inplace=True)

habt_df.habt.replace(to_replace=',', value='.', regex=True, inplace=True)
habt_df.habt = pd.to_numeric(habt_df.habt)
habt_df.habt = habt_df.habt.apply(lambda price: split_to(price, threshold=200, split=10))
habt_df.head()

,habt
date,
2019-08-05 16:56:00,100.63
2019-08-06 16:56:00,100.62
2019-08-07 16:56:00,100.72
2019-08-08 16:56:00,101.42
2019-08-09 16:56:00,101.31


In [5]:
# ---- MXRF11 Dataset ----
mxrf_df = pd.read_csv(f'{DATASETS_PATH}/Dataset - MXRF11.csv')
mxrf_df.rename(columns={"Date": "date", "Close": "mxrf"}, inplace=True)
mxrf_df.date = pd.to_datetime(mxrf_df.date, format='%d/%m/%Y %H:%M:%S')
mxrf_df.set_index(keys=['date'], inplace=True)

mxrf_df.mxrf.replace(to_replace=',', value='.', regex=True, inplace=True)
mxrf_df.mxrf = pd.to_numeric(mxrf_df.mxrf)
mxrf_df.mxrf = mxrf_df.mxrf.apply(lambda price: split_to(price, threshold=200, split=10))
mxrf_df.head()

,mxrf
date,
2016-01-04 16:56:00,8.18
2016-01-05 16:56:00,8.18
2016-01-06 16:56:00,8.18
2016-01-07 16:56:00,8.13
2016-01-08 16:56:00,8.14


In [6]:
# ---- VGHF11 Dataset ----
vghf_df = pd.read_csv(f'{DATASETS_PATH}/Dataset - VGHF11.csv')
vghf_df.rename(columns={"Date": "date", "Close": "vghf"}, inplace=True)
vghf_df.date = pd.to_datetime(vghf_df.date, format='%d/%m/%Y %H:%M:%S')
vghf_df.set_index(keys=['date'], inplace=True)

vghf_df.vghf.replace(to_replace=',', value='.', regex=True, inplace=True)
vghf_df.vghf = pd.to_numeric(vghf_df.vghf)
vghf_df.vghf = vghf_df.vghf.apply(lambda price: split_to(price, threshold=200, split=10))
vghf_df.head()

,vghf
date,
2021-03-09 16:56:00,9.90
2021-03-10 16:56:00,9.85
2021-03-11 16:56:00,9.85
2021-03-12 19:30:00,9.85
2021-03-15 16:56:00,9.85


In [7]:
# ---- Full Dataset ----
full_df = habt_df.merge(mxrf_df, on='date', how='left').merge(vghf_df, on='date', how='left')
full_df.fillna(method='ffill', inplace=True)
full_df.head()

,habt,mxrf,vghf
date,,,
2019-08-05 16:56:00,100.63,11.44,NaN
2019-08-06 16:56:00,100.62,11.15,NaN
2019-08-07 16:56:00,100.72,11.16,NaN
2019-08-08 16:56:00,101.42,10.96,NaN
2019-08-09 16:56:00,101.31,10.94,NaN


<h1 id='2-calculating-mean-returns-risks-volatilities' style='color:#7159c1; border-bottom:3px solid #7159c1; letter-spacing:2px; font-family:JetBrains Mono; font-weight: bold; text-align:left; font-size:240%;padding:0'>2 | Calculating Mean Returns, Risks and Volatilities</h1>

<h3 id='2.1-discrete-datas' style='color:#7159c1; letter-spacing:2px; font-family:JetBrains Mono; font-weight: bold; text-align:left; font-size:240%;padding:0'>2.1 | Discrete Datas</h3>

> **Discrete Return**

Used to calculate the Returns taking a block of period into consideration, such as days, weeks, months and years.

$$
\text{Rt} = \frac{\text{Pt}} {\text{P(t-1)}} - 1 = \frac{\text{Pt} - \text{P(t-1)}} {\text{P(t-1)}}
$$

where:

$\text{- Rt: Discrete Return}$

$\text{- Pt: Current Price}$

$\text{- P(t-1): Previous Price}$

---

> **Discrete Mean Return**

Used to calculate the Mean Return of Discrete Returns.

$$
\text{MRt} = \frac{\sum_{i=0}^{n} {(Rt(i))}} {n}
$$

where

$\text{- MRt: Mean Discrete Return}$

$\text{- RT: Discrete Return}$

$\text{- n: Number of Discrete Returns}$

---

> **Risk**

Used to calculate the `Standard Deviation` of Discrete Returns.

When dealing with Discrete Returns, the Standard Deviation is called `Risk`.

<h3 id='2.2-continuous-datas' style='color:#7159c1; letter-spacing:2px; font-family:JetBrains Mono; font-weight: bold; text-align:left; font-size:240%;padding:0'>2.2 | Continuous Datas</h3>

> **Continuous Returns**

Used to calculate the Returns taking all periods into consideration. For stocks, it's the daily or every moment of the day.

$$
\text{Rt} = \ln{(\frac{\text{Pt}} {\text{P(t-1)}})}
$$

where:

$\text{- Rt: Continuous Returns}$

$\text{- Pt: Current Price}$

$\text{- P(t-1): Previous Price}$

---

> **Continuous Mean Return**

Used to calculate the Mean Return of Continuous Returns.

$$
\text{MRt} = \frac{\sum_{i=0}^{n} {(\text{Rt(i)})}} {n}
$$

where:

$\text{- MRt: Continuous Mean Return}$

$\text{- Rt: Continuous Returns}$

$\text{- n: Number of Continuous Returns}$

---

> **Volatility**

Used to calculate the `Standard Deviation` of Continuous Returns.

When dealing with Continuous Returns, the Standard Deviation is called `Volatility`.

In [8]:
# ---- Calculating Discrete Returns ----
#
# - pct_change(): native Pandas function that calculates the Discrete Return
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.pct_change.html
#
discrete_returns_df = full_df.copy()
discrete_returns_df.habt = discrete_returns_df.habt.pct_change(periods=1)
discrete_returns_df.mxrf = discrete_returns_df.mxrf.pct_change(periods=1)
discrete_returns_df.vghf = discrete_returns_df.vghf.pct_change(periods=1)
discrete_returns_df.dropna(inplace=True)
discrete_returns_df.head()

,habt,mxrf,vghf
date,,,
2021-03-10 16:56:00,-0.007099,-0.008654,-0.005051
2021-03-11 16:56:00,0.004627,-0.001940,0.000000
2021-03-12 19:30:00,-0.001005,-0.006803,0.000000
2021-03-15 16:56:00,0.000168,0.000000,0.000000
2021-03-16 16:56:00,0.004693,-0.002935,0.000000


In [9]:
# ---- Calculating Continuous Returns ----
continuous_returns_df = full_df.copy()
continuous_returns_df.habt = np.log(continuous_returns_df.habt / continuous_returns_df.habt.shift(1))
continuous_returns_df.mxrf = np.log(continuous_returns_df.mxrf / continuous_returns_df.mxrf.shift(1))
continuous_returns_df.vghf = np.log(continuous_returns_df.vghf / continuous_returns_df.vghf.shift(1))
continuous_returns_df.dropna(inplace=True)
continuous_returns_df.head()

,habt,mxrf,vghf
date,,,
2021-03-10 16:56:00,-0.007125,-0.008692,-0.005063
2021-03-11 16:56:00,0.004616,-0.001942,0.000000
2021-03-12 19:30:00,-0.001005,-0.006826,0.000000
2021-03-15 16:56:00,0.000168,0.000000,0.000000
2021-03-16 16:56:00,0.004682,-0.002940,0.000000


<h1 id='3-exporting-datasets' style='color:#7159c1; border-bottom:3px solid #7159c1; letter-spacing:2px; font-family:JetBrains Mono; font-weight: bold; text-align:left; font-size:240%;padding:0'>3 | Exporting Datasets</h1>

In [10]:
with pd.ExcelWriter(f'{DATASETS_PATH}/Dataset - Full 2.xlsx') as writer:
    full_df.to_excel(writer, sheet_name='Dataset - Prices')
    discrete_returns_df.to_excel(writer, sheet_name='Dataset - Discrete Returns')
    continuous_returns_df.to_excel(writer, sheet_name='Dataset - Continuous Returns')

---

<h1 id='reach-me' style='color:#7159c1; border-bottom:3px solid #7159c1; letter-spacing:2px; font-family:JetBrains Mono; font-weight: bold; text-align:left; font-size:240%;padding:0'>📫 | Reach Me</h1>

> **Email** - [csfelix08@gmail.com](mailto:csfelix08@gmail.com?)

> **Linkedin** - [linkedin.com/in/csfelix/](https://www.linkedin.com/in/csfelix/)

> **GitHub:** - [CSFelix](https://github.com/CSFelix)

> **Kaggle** - [DSFelix](https://www.kaggle.com/dsfelix)

> **Portfolio** - [CSFelix.io](https://csfelix.github.io/).